# Event plots for Na-22

1. Run 3574, 3575
2. Threshold baseline 20 counts
3. Qmin = 90,0000 counts
4. width min > 7 mus
5. 5 PMTs / 11

In [1]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

2017-04-05 00:12:16


In [2]:
%gui qt
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
import time
import tables as tb
import numpy as np

In [4]:
from invisible_cities.database import load_db

In [5]:
import invisible_cities.reco.tbl_functions as tbl
import invisible_cities.reco.wfm_functions as wfm
import invisible_cities.sierpe.blr as blr
import invisible_cities.reco.peak_functions_c as cpf
from invisible_cities.core.mpl_functions import set_plot_labels
from invisible_cities.reco.params import S12Params, ThresholdParams
from   invisible_cities.core.system_of_units_c import units

In [6]:
from gui_functions import fplot_pmt_waveforms, fplot_pmt_signals_vs_time_mus, fplot_signal_vs_time_mus,\
                           fplot_s12, hist_1d

In [7]:
from event_pmaps import EventPmaps, print_s12, print_s2si

In [74]:
run_number = 3575

In [75]:
DataPMT = load_db.DataPMT(run_number)

In [76]:
from PyQt5 import QtCore, QtWidgets, uic
from matplotlib.backends.backend_qt5agg import (
    FigureCanvasQTAgg as FigureCanvas,
    NavigationToolbar2QT as NavigationToolbar)
from matplotlib.figure import Figure

from PyQt5.uic import loadUiType


qtCreatorFile = "event_maps.ui" # Enter file here.
Ui_MainWindow, QMainWindow = loadUiType(qtCreatorFile)

In [77]:
class QtGui(QMainWindow, Ui_MainWindow):
    def __init__(self, ):
        super(QtGui, self).__init__()
        self.setupUi(self)
        
        self.fig_dict = {}
        self.mplfigs.itemClicked.connect(self.change_figure)

        fig = Figure()
        self.add_mpl(fig)

    def add_figure(self, name, fig):
        test_key = self.fig_dict.pop(name, None)
        self.fig_dict[name] = fig
        if not test_key: # key not in dict
            self.mplfigs.addItem(name)

    def change_figure(self, item):
        text = item.text()
        self.rm_mpl()
        self.add_mpl(self.fig_dict[text])

    def add_mpl(self, fig):
        self.canvas = FigureCanvas(fig)
        self.mplvl.addWidget(self.canvas)
        self.canvas.draw()
        self.toolbar = NavigationToolbar(self.canvas,
                self.mplwindow, coordinates=True)
        self.mplvl.addWidget(self.toolbar)

    def rm_mpl(self,):
        self.mplvl.removeWidget(self.canvas)
        self.canvas.close()
        self.mplvl.removeWidget(self.toolbar)
        self.toolbar.close()



In [78]:
def scan_events(epm, gui, pmtrwf, sipmrwf, 
                event_list=list(range(1)), 
                event_plot_list=list(range(1)), 
                printout=10):
    """Loop over events"""
    
    for event in event_list:
        if(event%printout == 0):
            print('++++++processing event number {}'.format(event))
        epm.calibrated_pmt_and_csum(event, pmtrwf)
        epm.calibrated_sipm(event, sipmrwf)
        
        if(event%printout == 0):
            print('--S1 search')
        epm.find_s1()
        
        if(event%printout == 0):
            print('--S2 search')
        epm.find_s2()
        
        if (len(epm.S2) > 0):
            epm.find_s2si()
        
        if event in event_plot_list:
            gui.add_figure('RWF-event-{}'.format(event), 
                            fplot_pmt_waveforms(epm.RWF, zoom=False, window_size=10000))
            gui.add_figure('CWF_vs_time_mus-event-{}'.format(event), 
                            fplot_pmt_signals_vs_time_mus(epm.CWF,
                                                         epm.P.pmt_active,
                                                         t_min      =    0,
                                                         t_max      = 1300,
                                                         signal_min =    -5,
                                                         signal_max =  200))
            gui.add_figure('Calibrated_SUM-event-{}'.format(event), 
                            fplot_signal_vs_time_mus(epm.csum, t_min=0, t_max=1300, signal_min=-5, signal_max=100))
            if (len(epm.S1) > 0):
                gui.add_figure('S1-event-{}'.format(event), fplot_s12(epm.S1))
            if (len(epm.S2) > 0):
                gui.add_figure('S2-event-{}'.format(event), fplot_s12(epm.S2))
                

In [79]:
RWF_file = os.path.join(os.environ['IC_DATA'],
                        'LSC/wvfms/{}/run_{}.gdc1next.next-new.000.rd_{}.root.h5'.format(run_number, 
                                                                                         run_number,
                                                                                         run_number))


In [80]:
h5rwf = tb.open_file(RWF_file,'r')

In [81]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [82]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))


        Number of events in file = 9
        Number of PMTs = 12
        PMTWL = 52000
        Number of SiPMs = 1792
        SiPMWL = 1300
      


In [83]:
s1par  = S12Params(tmin=0*units.mus, tmax=640*units.mus, lmin=8, lmax=20, stride=4, rebin=False)
s2par = S12Params(tmin=645*units.mus, tmax=1300*units.mus, stride=80, lmin=80, lmax=20000, rebin=True)
thr    = ThresholdParams(thr_s1=3 * units.pes,  thr_s2=5 *units.pes,
                        thr_MAU = 3 * units.adc, thr_sipm = 3.5 * units.pes,
                        thr_SIPM = 30 * units.adc)

In [84]:
gui=QtGui()
gui.show()

In [85]:
epm = EventPmaps(run_number, s1par, s2par, thr, verbose=True)

### Run event 0

In [86]:
event=0
epm.calibrated_pmt_and_csum(event, pmtrwf)
epm.calibrated_sipm(event, sipmrwf)
epm.find_s1()
epm.find_s2()
if (len(epm.S2) > 0):
    epm.find_s2si()

number of peaks = 1
S12 number = 0, samples = 13 sum in pes =368.76143716367665
time vector (mus) = [ 486.025  486.05   486.075  486.1    486.125  486.15   486.175  486.2
  486.225  486.25   486.275  486.3    486.325]
energy vector (pes) = [  6.06866812  21.31666428  42.70261378  59.5391848   64.29549092
  56.63435955  43.18807276  29.15238963  17.98361241  11.19797441
   7.9264976    5.03344965   3.72245928]
number of peaks = 2
S12 number = 0, samples = 36 sum in pes =134461.73503658187
time vector (mus) = [ 641.48875     646.4625      647.4625      648.4625      649.4625
  650.4625      651.4625      652.4625      653.4625      654.4625
  655.4625      656.4625      657.4625      658.4625      659.4625
  660.4625      661.4625      662.4625      663.4625      664.4625
  665.4625      666.4625      667.4625      668.4625      669.4625
  670.4625      671.4625      672.4625      673.4625      674.4625
  675.4625      676.4625      677.4625      680.119375    682.05375
  683.15416667]
e

In [87]:
gui.add_figure('RWF-event-{}'.format(event), 
                            fplot_pmt_waveforms(epm.RWF, zoom=False, window_size=10000))

In [88]:
gui.add_figure('CWF-event-{}'.format(event), 
                fplot_pmt_signals_vs_time_mus(epm.CWF,
                                             epm.P.pmt_active,
                                             t_min      =    0,
                                             t_max      = 1300,
                                             signal_min =    -5,
                                             signal_max =  250))

In [89]:
gui.add_figure('PMTs-event-{}'.format(event), 
                fplot_pmt_signals_vs_time_mus(epm.CAL_PMT,
                                             epm.P.pmt_active,
                                             t_min      =    0,
                                             t_max      = 1300,
                                             signal_min =    -5,
                                             signal_max =  10))

In [90]:
gui.add_figure('Calibrated_SUM-event-{}'.format(event), 
                            fplot_signal_vs_time_mus(epm.csum, t_min=0, t_max=1300, signal_min=-5, signal_max=100))

In [91]:
if (len(epm.S1) > 0):
    gui.add_figure('S1-event-{}'.format(event), fplot_s12(epm.S1))
if (len(epm.S2) > 0):
    gui.add_figure('S2-event-{}'.format(event), fplot_s12(epm.S2))

### Run over the file

In [92]:
n_events = 9
sample_plot = 1
t0 = time.time()
scan_events(epm, gui, pmtrwf, sipmrwf, 
            event_list=list(range(n_events)), 
            event_plot_list=list(range(0,n_events,sample_plot)),
           printout=1)
t1 = time.time()        
dt = t1 - t0
print("run {} events in {} s event/s = {}".format(n_events, dt, dt/n_events))

++++++processing event number 0
--S1 search
number of peaks = 1
S12 number = 0, samples = 13 sum in pes =368.76143716367665
time vector (mus) = [ 486.025  486.05   486.075  486.1    486.125  486.15   486.175  486.2
  486.225  486.25   486.275  486.3    486.325]
energy vector (pes) = [  6.06866812  21.31666428  42.70261378  59.5391848   64.29549092
  56.63435955  43.18807276  29.15238963  17.98361241  11.19797441
   7.9264976    5.03344965   3.72245928]
--S2 search
number of peaks = 2
S12 number = 0, samples = 36 sum in pes =134461.73503658187
time vector (mus) = [ 641.48875     646.4625      647.4625      648.4625      649.4625
  650.4625      651.4625      652.4625      653.4625      654.4625
  655.4625      656.4625      657.4625      658.4625      659.4625
  660.4625      661.4625      662.4625      663.4625      664.4625
  665.4625      666.4625      667.4625      668.4625      669.4625
  670.4625      671.4625      672.4625      673.4625      674.4625
  675.4625      676.4625     